In [2]:
import pandas as pd
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
import os 

### Ring buffer implementation

For behaviors: Gå gjennom en sortert liste med data (radene skal være sortert på tid). Så gå gjennom impressions-listen. Der det er en -"1", legg til artikkelens ID i ring-buffer (dette er en artikkel som brukeren har trykket på). 

Så ser vi på neste bruker/rad, som nå er ukjent, siden det er utenfor tidsvinduet. Vi ser tilbake n plsser i bufferet, og gir dette som impressions til denne brukeren. Deretter sjekker vi om det denne brukeren faktisk hadde en 1er på i sin impressionliste var noen av artiklene vi anbefalte. 

Et problem: vi vet ikke hvilken nettavis brukerne tilhører, og vi ikek hvordan impression-listen de faktisk fikk ble generert. To brukere kan komme etter hverandre på listen av brukere når vi sorterer på tid, og lese artikler på to helt forskjellige nettaviser. Dette vil i såfall gi lav score på evalueringen.

In [3]:
#path to the mindsmall training data
training_dataset = '../MINDsmall_train'

validation_dataset = '../MINDsmall_train'

entity_embedding = pd.read_csv("../MINDsmall_train/entity_embedding.vec", sep='\t')
relation_embedding = pd.read_csv("../MINDsmall_train/relation_embedding.vec", sep='\t')


news_data = pd.read_csv("../MINDsmall_train/news.tsv",
    sep='\t',
    names=["newsId", "category", "subcategory", "title","abstract", "url", "title_entities","abstract_entities"]
)

behaviors_data = pd.read_csv(
    "../MINDsmall_train/behaviors.tsv",
    sep='\t',
    names=["impressionId", "userId", "timestamp", "click_history", "impressions"],
    parse_dates=['timestamp'] 
)

### Preprocess data

In [4]:
behaviors_data['timestamp'] = pd.to_datetime(behaviors_data['timestamp'], format='%Y-%m-%d %H:%M:%S')
behaviors_data['clicks'] = behaviors_data['click_history'].str.split().str.len()
behaviors_data['impressions_list'] = behaviors_data['impressions'].str.split()
behaviors_data['impressions_count'] = behaviors_data['impressions_list'].str.len()

# remove null values
news_data = news_data.dropna().reset_index(drop=True)


### Manglende verdier for clicks og click_history

In [5]:
# missing_values_news = news_data.isnull().sum()
# print("Missing values in news_data:")
# print(missing_values_news)


# missing_values_behaviors = behaviors_data.isnull().sum()
# print("\nMissing values in behaviors_data:")
# print(missing_values_behaviors)

In [6]:
null_clicks_rows = behaviors_data[behaviors_data['clicks'].isnull() & behaviors_data['click_history'].isnull()]

# Display the rows
print("Rows with null values for clicks and click_history:")
print(null_clicks_rows)

Rows with null values for clicks and click_history:
        impressionId  userId           timestamp click_history  \
28                29  U33207 2019-11-11 11:09:14           NaN   
96                97  U89839 2019-11-11 04:57:51           NaN   
155              156   U5598 2019-11-13 14:01:40           NaN   
194              195  U19208 2019-11-11 15:57:34           NaN   
257              258   U6879 2019-11-14 17:33:52           NaN   
...              ...     ...                 ...           ...   
156850        156851  U85936 2019-11-10 08:52:02           NaN   
156890        156891   U4828 2019-11-11 09:41:21           NaN   
156897        156898  U38290 2019-11-14 15:50:07           NaN   
156921        156922  U64209 2019-11-11 07:11:07           NaN   
156935        156936  U35188 2019-11-11 13:39:46           NaN   

                                              impressions  clicks  \
28      N62212-0 N27521-0 N28983-0 N12028-0 N25437-0 N...     NaN   
96               

In [7]:
news_data.head()

,newsId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [8]:
behaviors_data.head()

,impressionId,userId,timestamp,click_history,impressions,clicks,impressions_list,impressions_count
0,1,U13740,2019-11-11 09:05:58,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,9.0,"[N55689-1, N35729-0]",2
1,2,U91836,2019-11-12 18:11:30,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,82.0,"[N20678-0, N39317-0, N58114-0, N20495-0, N4297...",11
2,3,U73700,2019-11-14 07:01:48,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,16.0,"[N50014-0, N23877-0, N35389-0, N49712-0, N1684...",36
3,4,U34670,2019-11-11 05:28:05,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,10.0,"[N35729-0, N33632-0, N49685-1, N27581-0]",4
4,5,U8125,2019-11-12 16:11:21,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,4.0,"[N39985-0, N36050-0, N16096-0, N8400-1, N22407...",69


In [9]:
# sort behaviors_data by timestamp
behaviors_data = behaviors_data.sort_values(by='timestamp')

In [10]:
behaviors_data.head()

,impressionId,userId,timestamp,click_history,impressions,clicks,impressions_list,impressions_count
20111,20112,U65916,2019-11-09 00:00:19,N51706 N40767 N12096 N9798 N38802 N54827 N5780...,N54300-0 N46057-1 N57005-0 N52154-0 N57099-0 N...,15.0,"[N54300-0, N46057-1, N57005-0, N52154-0, N5709...",31
13806,13807,U49985,2019-11-09 00:01:13,N5056 N29975 N53234 N39603 N50032 N8422 N53580...,N20602-0 N50059-0 N57768-1 N50135-1 N15134-0 N...,93.0,"[N20602-0, N50059-0, N57768-1, N50135-1, N1513...",29
27659,27660,U25550,2019-11-09 00:02:44,N17260 N38298 N33976 N47719 N14888 N18870 N4607,N50135-0 N15134-0 N52433-1 N20602-0 N64536-0,7.0,"[N50135-0, N15134-0, N52433-1, N20602-0, N6453...",5
152216,152217,U19710,2019-11-09 00:02:50,N3530 N48284 N43019 N62546 N138 N13138 N10676 ...,N57099-0 N30295-0 N21086-0 N5379-0 N57005-0 N4...,8.0,"[N57099-0, N30295-0, N21086-0, N5379-0, N57005...",56
42165,42166,U38106,2019-11-09 00:03:09,N16874 N264 N48697 N51366,N3491-0 N20602-0 N25785-0 N23575-0 N38783-0 N1...,4.0,"[N3491-0, N20602-0, N25785-0, N23575-0, N38783...",48


### Ring buffer

In [11]:
class RingBuffer:
    """ Ring buffer class. """
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.impressions = []

    class __Full:
        """ Full buffer."""
        def add(self, x):
            """ Append an element overwriting the oldest one."""
            self.impressions[self.pointer] = x
            self.pointer = (self.pointer+1) % self.buffer_size

        def get(self):
            """ Return list of elements in correct order."""
            return self.impressions[self.pointer:] + self.impressions[:self.pointer]

    def add(self,x):
        """ Add impression to the end of the list."""
        self.impressions.append(x)
        if len(self.impressions) == self.buffer_size:
            self.pointer = 0
            # from now on, self.__class__ refers to the __Full class
            self.__class__ = self.__Full

    def get(self):
        """ Returns the list of impressions."""
        return self.impressions


### Add a user's impressions to the buffer

We go through the dataframe that is sorted on the timestamps. The ring buffer is instantiated, and we don't start recommending articles until we have gone through a couple of users so that the buffer has a couple of articles in it. Then we check if the recommendation from the ring buffer match up with the actual impression list for the rest of the users. 

In [137]:
wait_count = 10
max_users = wait_count + 200
last_user = 0
ring_buffer = RingBuffer(20)

actual_impressions = []
recommended_impressions = []

for index, row in behaviors_data.iterrows():

    user_impressions = []

    if row['clicks'] > 0 and max_users > 0:
        last_user = index
        #the articles that the user clicked on
        [user_impressions.append(n.split("-")[0]) for n in row['impressions_list'] if n.split("-")[1] == "1"]

        # we wait for users to click on articles before we start recommending articles
        if wait_count == 0:
            recommended_impressions.append((index, set(ring_buffer.get())))
            actual_impressions.append((index, set(user_impressions)))

        #add the current user clicks to the buffer
        for impression in user_impressions:
            ring_buffer.add(impression)

    wait_count -= 1 if wait_count > 0 else 0
    # we can remove this condition to get all the users
    max_users -= 1
    if max_users == 0:
        break

### Evaluate the Ring Buffer

In [140]:
def evaluate_ring_buffer(actual, recommended):
    total_accuracy = 0
    print(f'Number of entries evaluated: {len(actual)}')

    for i in range(len(actual)):
        intersection = actual[i][1].intersection(recommended[i][1])
        num_matches = len(intersection)

        total_accuracy += num_matches

    print(f'Total accuracy: {(total_accuracy / len(actual))*100:.2f}%')


evaluate_ring_buffer(actual_impressions, recommended_impressions)

Number of entries evaluated: 199
Total accuracy: 45.73%
